In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
url = URL(
    user="ABUHTAREVICH",
    password="",
    account=".us-.gcp",
    warehouse="COMPUTE_WH",
    database="",
    schema="",
    #role='xxxxx',
    #authenticator='https://xxxxx.okta.com',
)
engine = create_engine(url)
connection = engine.connect()

In [3]:
# show all tables

pd.set_option('display.max_rows', 500)

In [4]:
query = '''
with 
users_installs as (
select 
    adid
from events
where app_id = '1636821805'
    and to_date(created_at) between '2023-01-06' and '2023-01-23'
    and activity_kind = 'install'
    and country = 'us'
    and app_version_short in ('1.8', '1.9')
),

user_start as (
select
    adid
from events
where app_id = '1636821805'
and datediff(day, to_date(installed_at), to_date(created_at)) between 0 and 25
and adid in (select adid from users_installs)
and event_name in ('game_start')
and parse_json(publisher_parameters):lvl_total::integer = 35
),

user_end as (
select
    adid
from events
where app_id = '1636821805'
and datediff(day, to_date(installed_at), to_date(created_at)) between 0 and 25
and adid in (select adid from user_start)
and event_name in ('game_win')
and parse_json(publisher_parameters):lvl_total::integer = 35
)

select
    adid,
    created_at,
    event_name,
    parse_json(publisher_parameters):lvl_total::integer lvl_total,
    parse_json(publisher_parameters):themeName::string theme,
    parse_json(publisher_parameters):lvl::integer lvl,
    parse_json(publisher_parameters):count_word_fail::integer count_word_fail,
    parse_json(publisher_parameters):type::string type,
    parse_json(publisher_parameters):answer::string answer,
    parse_json(publisher_parameters):result::string result,
    parse_json(publisher_parameters):time_round::integer time_round,
    parse_json(publisher_parameters):time_round::integer time_round2
from events
where app_id = '1636821805'
and datediff(day, to_date(installed_at), to_date(created_at)) between 0 and 25
and adid in (select adid from user_start)
and adid not in (select adid from user_end)
and event_name in ('game_word_end', 'game_hint_spend', 'game_word_fail')
and parse_json(publisher_parameters):lvl_total::integer = 35
'''

In [5]:
df = pd.read_sql(query, connection)

In [6]:
df.dtypes

adid               object
created_at         object
event_name         object
lvl_total           int64
theme              object
lvl                 int64
count_word_fail    object
type               object
answer             object
result             object
time_round         object
time_round2        object
dtype: object

In [7]:
df.isna().sum()

adid                  0
created_at            0
event_name            0
lvl_total             0
theme                 0
lvl                   0
count_word_fail    1822
type                  0
answer                0
result              334
time_round         1822
time_round2        1822
dtype: int64

In [8]:
df.head()

,adid,created_at,event_name,lvl_total,theme,lvl,count_word_fail,type,answer,result,time_round,time_round2
0,35b9c355a12e29f6e16162f169d975b3,1673469741,game_hint_spend,35,Mathematics,1,None,words,FUTURE,None,None,None
1,0d3cc82690a322d4eb148b9f737a0dd3,1674801704,game_word_fail,35,Mathematics,1,None,words,CLOSE,HUMID,None,None
2,0d3cc82690a322d4eb148b9f737a0dd3,1674801782,game_hint_spend,35,Mathematics,1,None,words,COLLEGE,None,None,None
3,0d3cc82690a322d4eb148b9f737a0dd3,1674801987,game_word_fail,35,Mathematics,1,None,words,CALL,FALL,None,None
4,0d3cc82690a322d4eb148b9f737a0dd3,1674801991,game_word_fail,35,Mathematics,1,None,words,CALL,BULL,None,None


In [9]:
df.adid.nunique()

295

In [14]:
df.query('type == "words"') \
        .groupby(['event_name'], as_index=False).agg({'adid': 'nunique', 'lvl_total': 'count'})

,event_name,adid,lvl_total
0,game_hint_spend,76,115
1,game_word_end,81,175
2,game_word_fail,250,1038


In [15]:
81/500

0.162

In [ ]:
df[df.lvl_total.notnull()].event_name.value_counts()

In [ ]:
df.created_at = df.created_at.astype('int')

In [ ]:
df.adid.unique()

In [ ]:
df.query('adid == "9cc07c6869448aaa05d7c87d9a66846f"')

In [ ]:
df.query('event_name in ("ads_interstitial_potential", "ads_interstitial_impression")') \
    .groupby(['event_name', 'placement'], as_index=False) \
    .agg({'adid': 'nunique', 'created_at': 'count'}) \
    .pivot(index='event_name', columns='placement', values=['adid', 'created_at'])

In [ ]:
df.adid.nunique()

In [ ]:
1918-359

In [ ]:
1559/2257

In [ ]:
df_time_1_lvl = df.query('event_name == "game_win" & lvl_total == 2 & theme == "Interesting"') \
        .groupby('adid', as_index=False) \
        .agg({'created_at': max}) \
        .rename(columns={'created_at': 'time_1_lvl'})

In [ ]:
df_lvl = df_time_1_lvl.merge(df, how='left', on='adid')

In [ ]:
all_users = df_lvl.adid.nunique()
all_users

In [ ]:
df_lvl.adid.unique()

In [ ]:
0.871954-0.136464

In [ ]:
df_on_1_lvl = df_lvl.query('created_at >= time_1_lvl').copy()

df_events_on_1_lvl = df_on_1_lvl.groupby('event_name', as_index=False) \
           .agg({'adid': 'nunique', 'created_at': 'count'}) \
           .rename(columns={'created_at': 'events_counts'}) \

df_events_on_1_lvl['event_per_user'] = df_events_on_1_lvl.events_counts / df_events_on_1_lvl.adid
df_events_on_1_lvl['conversion'] = df_events_on_1_lvl.adid / all_users

df_events_on_1_lvl = df_events_on_1_lvl.sort_values('adid', ascending=False)

df_events_on_1_lvl.query(
    'event_name not in ("game_start", "ads_revenue_threshold", "revenue_threshold", "ab_test", "onboarding_game_1_end", "game_win")'
)[['event_name', 'conversion', 'event_per_user']]

In [ ]:
df_pontential = df.query('event_name == "ads_interstitial_potential"') \
                    .groupby('placement', as_index=False) \
                    .agg({'adid': 'nunique', 'created_at': 'count'}) \
                    .rename(columns={'created_at': 'events_counts'}) \

df_pontential['event_per_user'] = df_pontential.events_counts / df_pontential.adid

df_pontential['conversion'] = df_pontential.adid / all_users

df_pontential = df_pontential.sort_values('adid', ascending=False)

df_pontential[['placement', 'conversion', 'event_per_user']]

In [ ]:
df_impression = df.query('event_name == "ads_interstitial_impression"') \
                    .groupby('placement', as_index=False) \
                    .agg({'adid': 'nunique', 'created_at': 'count'}) \
                    .rename(columns={'created_at': 'events_counts'}) \

df_impression['event_per_user'] = df_impression.events_counts / df_impression.adid

df_impression['conversion'] = df_impression.adid / all_users

df_impression = df_impression.sort_values('adid', ascending=False)

df_impression[['placement', 'conversion', 'event_per_user']]

In [ ]:
df_word_end = df.query('event_name == "game_word_end" & lvl_total == 2 & theme == "Interesting"') \
    .groupby('answer', as_index=False) \
    .agg({'adid': 'nunique'})

df_word_end['per_all'] = df_word_end.adid / all_users

df_word_end = df_word_end.sort_values('adid', ascending=False)

df_word_end[['answer', 'per_all']]

In [ ]:
df_word_fail = df.query('event_name == "game_word_fail" & lvl_total == 2 & theme == "Interesting"') \
    .groupby('answer', as_index=False) \
    .agg({'adid': 'nunique', 'event_name': 'count'})

df_word_fail['per_all'] = df_word_fail.adid / all_users
df_word_fail['per_user'] = df_word_fail.event_name / df_word_fail.adid

df_word_fail = df_word_fail.sort_values('adid', ascending=False)

df_word_fail[['answer', 'per_all', 'per_user']]

In [ ]:
df_hint_spend = df.query('event_name == "game_hint_spend" & lvl_total == 2 & theme == "Interesting"') \
    .groupby('answer', as_index=False) \
    .agg({'adid': 'nunique', 'event_name': 'count'})

df_hint_spend['per_all'] = df_hint_spend.adid / all_users
df_hint_spend['per_user'] = df_hint_spend.event_name / df_hint_spend.adid

df_hint_spend = df_hint_spend.sort_values('adid', ascending=False)

df_hint_spend[['answer', 'per_all', 'per_user']]

In [ ]:
df_time_max = df_lvl.groupby(['adid', 'time_2_lvl'], as_index=False) \
                .agg({'created_at': max}) \
                .rename(columns={'created_at': 'max_time'})

df_time_max['time_diff'] = df_time_max.max_time - df_time_max.time_2_lvl

In [ ]:
df_time_max.time_diff.describe()

In [ ]:
df_time_max.time_diff.quantile(0.35)

In [ ]:
df_time_max.time_diff.quantile(0.45)

In [ ]:
df_time_max.time_diff.quantile(0.5)

In [ ]:
df_time_max.time_diff.quantile(0.7)

In [ ]:
plt.figure(figsize=(8, 4), dpi=80)

plt.title('Распределение 70% самых малоактивных юзеров по времени, после старта 2 уровня', fontsize=14)
plt.xlabel('Время, сек', fontsize=12)
plt.ylabel('Кол-во юзеров', fontsize=12)
sns.histplot(data=df_time_max.query('time_diff < 115').time_diff, 
            shrink=.85);

In [ ]:
df_word_action = df.query('event_name == "game_word_action"') \
                                      .groupby(['lvl_total', 'theme', 'lvl'], as_index=False) \
                                      .agg({'adid': 'nunique', 'event_name': 'count'}) \
                                      .rename(columns={'adid': 'users_word_action'}) \
                                      .sort_values('users_word_action', ascending=False)

In [ ]:
df_game_word_end_word = df.query('event_name == "game_word_end" & type == "words"') \
                                      .groupby(['lvl_total', 'theme', 'lvl'], as_index=False) \
                                      .agg({'event_name': 'count'}) \
                                      .rename(columns={'event_name': 'events_words'})

In [ ]:
df_game_word_end_solution = df.query('event_name == "game_word_end" & type == "solution"') \
                                      .groupby(['lvl_total', 'theme', 'lvl'], as_index=False) \
                                      .agg({'event_name': 'count'}) \
                                      .rename(columns={'event_name': 'events_solution'})

In [ ]:
df_word_solution = df_game_word_end_word.merge(df_game_word_end_solution, how='outer').fillna(0)

In [ ]:
df_word_solution['word_per_solution'] = df_word_solution.events_solution / (
                                                df_word_solution.events_words + df_word_solution.events_solution)

In [ ]:
df_word_solution = df_word_solution[['lvl_total', 'theme', 'lvl', 'word_per_solution']].copy()

In [ ]:
df_lvl_word_solution = n_lvls.merge(df_word_solution, how='outer').fillna(0)
df_lvl_word_solution = df_lvl_word_solution[['N', 'theme', 'lvl', 'word_per_solution']]
df_lvl_word_solution.head()

In [ ]:
# финальный DF - df_final_last - его копировал в гугл шит и некоторые колонки добавлял руками по аналогии можно смотреть как 
# в последнем репорте

In [ ]:
df_final_last

In [ ]:
df_final

In [ ]:
df_final_last

In [ ]:
df.query('event_name == "ab_test"')

In [ ]:
df.query('lvl_total == 3')

In [ ]:
df_lvl = df.merge(n_lvls)

In [ ]:
# ввместо N выбирать нужный лвл того, чтоб посмотреть по словам на уровне

In [ ]:
df_3_lvl = df.query('lvl_total == 3')

In [ ]:
start_lvl_users = df_3_lvl.query('event_name == "game_start"').adid.nunique()

In [ ]:
events_list = ('game_hint_spend', 'game_word_action', 'game_word_end', 'game_word_fail')

In [ ]:
df_3_lvl_words = df_3_lvl.query('event_name in @events_list & type == "words"') \
    .groupby(['answer', 'event_name'], as_index=False) \
    .agg({'adid': 'nunique', 'theme': 'count'}) \
    .rename(columns={'adid': 'users_nunique', 'theme': 'total_events'})

df_3_lvl_words['events_per_user'] = round(df_3_lvl_words.total_events / df_3_lvl_words.users_nunique, 2)
df_3_lvl_words['users_per_users_win'] = df_3_lvl_words.users_nunique / start_lvl_users
df_3_lvl_words = df_3_lvl_words[['answer', 'event_name', 'users_per_users_win', 'events_per_user']]

In [ ]:
df_3_lvl_words.set_index(['answer', 'event_name']) \
                .users_per_users_win.unstack().reset_index()

In [ ]:
df_3_lvl_words_uniq_users = df_3_lvl_words.set_index(['answer', 'event_name']) \
                .users_per_users_win.unstack().reset_index()
df_3_lvl_words_uniq_users.columns = ('answer', 'hint_users', 'action_users', 'end_users', 'fail_users')

In [ ]:
df_3_lvl_words_per_user = df_3_lvl_words.set_index(['answer', 'event_name']) \
                .events_per_user.unstack().reset_index()
df_3_lvl_words_per_user.columns = ('answer', 'hint_events', 'action_events', 'end_events', 'fail_events')

In [ ]:
df_3_lvl_words_total = df_3_lvl_words_uniq_users.merge(df_3_lvl_words_per_user)

In [ ]:
df_3_lvl_words_total = df_3_lvl_words_total[['answer', 'fail_users', 'fail_events', 'end_users', 'hint_users', 
                      'hint_events', 'action_users', 'action_events']]

In [ ]:
df_3_lvl_top_fail = df_3_lvl.query('event_name == "game_word_fail" & type == "words"') \
                .groupby(['answer', 'result'], as_index=False) \
                .agg({'adid': 'count'})

In [ ]:
max_fails = df_3_lvl_top_fail.groupby('answer', as_index=False) \
                 .agg({'adid': max})

In [ ]:
df_3_lvl_max_fails_per_answer = df_3_lvl_top_fail.merge(max_fails, on=['answer', 'adid'])
df_3_lvl_max_fails_per_answer = df_3_lvl_max_fails_per_answer[['answer', 'result']]

In [ ]:
df_3_lvl_words_final = df_3_lvl_words_total.merge(df_3_lvl_max_fails_per_answer)

In [ ]:
df_3_lvl_words_final.sort_values('fail_users', ascending=False)